In [1]:
import pandas as pd

In [2]:
data_path = './raw_datasets/electronics.inter'

df = pd.read_csv(data_path, sep='\t')

In [3]:
df = df.rename(columns={"user_id:token": "session_id", "item_id:token": "item_id", "rating:float": "rating", "timestamp:float": "timestamp"})

In [4]:
df.sort_values(by=['session_id','timestamp'], ignore_index=True)

,session_id,item_id,rating,timestamp
0,A00000262KYZUE4J55XGL,B003UYU16G,5.0,1353456000
1,A000063614T1OE0BUSKUT,B00419ZT3E,5.0,1365120000
2,A000063614T1OE0BUSKUT,B00432ZSHG,5.0,1365120000
3,A00009182QVLSWIGHLS1B,B009SXR7WE,5.0,1364947200
4,A00009661LC9LQPGKJ24G,B004GWQBWY,5.0,1351209600
...,...,...,...,...
7824477,AZZZRS1YZ8HVP,B001CJOLBW,4.0,1317600000
7824478,AZZZRS1YZ8HVP,B00DR0PDNE,4.0,1389398400
7824479,AZZZSIK7NFFVP,B009FU8BR0,5.0,1385942400
7824480,AZZZWXXUPZ1F3,B0016CFZQ0,5.0,1358035200


In [5]:
df.groupby("session_id").count().describe()

,item_id,rating,timestamp
count,4.201696e+06,4.201696e+06,4.201696e+06
mean,1.862220e+00,1.862220e+00,1.862220e+00
std,2.885110e+00,2.885110e+00,2.885110e+00
min,1.000000e+00,1.000000e+00,1.000000e+00
25%,1.000000e+00,1.000000e+00,1.000000e+00
50%,1.000000e+00,1.000000e+00,1.000000e+00
75%,2.000000e+00,2.000000e+00,2.000000e+00
max,5.200000e+02,5.200000e+02,5.200000e+02


In [6]:
df['tmp'] = 1.0

In [7]:
df.loc[df.rating < 4.0, 'tmp'] = 0.0

In [8]:
df = df.drop(columns='rating').rename(columns={'tmp': 'rating'})

In [9]:
len(df[df.rating == 1]), len(df)

(5833322, 7824482)

In [10]:
df.session_id.nunique(), df.item_id.nunique(), len(df)

(4201696, 476002, 7824482)

In [11]:
df.describe()

,timestamp,rating
count,7.824482e+06,7.824482e+06
mean,1.338178e+09,7.455218e-01
std,6.900426e+07,4.355675e-01
min,9.127296e+08,0.000000e+00
25%,1.315354e+09,0.000000e+00
50%,1.361059e+09,1.000000e+00
75%,1.386115e+09,1.000000e+00
max,1.406074e+09,1.000000e+00


In [12]:
# df.to_csv("./Data/electronics_new.csv", index=False)

In [23]:
# sample
import numpy as np

np.random.seed(2022)

tot_item = df.item_id.nunique()

n_sampled_item = 300000

rand_id = df.item_id.unique()[np.random.choice(range(tot_item), n_sampled_item, replace=False)]

sampled_data = df[df.item_id.isin(rand_id)].reset_index(drop=True)

In [24]:
sampled_data.groupby("session_id").count().describe()

,item_id,timestamp,rating
count,3.010976e+06,3.010976e+06,3.010976e+06
mean,1.630826e+00,1.630826e+00,1.630826e+00
std,2.124423e+00,2.124423e+00,2.124423e+00
min,1.000000e+00,1.000000e+00,1.000000e+00
25%,1.000000e+00,1.000000e+00,1.000000e+00
50%,1.000000e+00,1.000000e+00,1.000000e+00
75%,2.000000e+00,2.000000e+00,2.000000e+00
max,3.400000e+02,3.400000e+02,3.400000e+02


In [25]:
# sampled_data.to_csv("Data/electronics_sampled.csv", index=False)

In [26]:
# df.to_csv("Data/electronics.csv", index=False)

In [27]:
import logging
from pathlib import Path
import pickle
from os import path

import pandas as pd
from sklearn.utils import shuffle

from configuration.config import *

from dataloaders.utils import _drop_cold

In [28]:
df

,session_id,item_id,timestamp,rating
0,AKM1MP6P0OYPR,0132793040,1365811200,1.0
1,A2CX7LUOHB2NDG,0321732944,1341100800,1.0
2,A2NWSAGRHCP8N5,0439886341,1367193600,0.0
3,A2WNBOD3WNDNKT,0439886341,1374451200,0.0
4,A1GI0U4ZRJA8WN,0439886341,1334707200,0.0
...,...,...,...,...
7824477,A2YZI3C9MOHC0L,BT008UKTMW,1396569600,1.0
7824478,A322MDK0M89RHN,BT008UKTMW,1313366400,1.0
7824479,A1MH90R0ADMIK0,BT008UKTMW,1404172800,1.0
7824480,A10M2KEFPEQDHN,BT008UKTMW,1297555200,1.0


In [29]:
sampled_data

,session_id,item_id,timestamp,rating
0,A2CX7LUOHB2NDG,0321732944,1341100800,1.0
1,A2NWSAGRHCP8N5,0439886341,1367193600,0.0
2,A2WNBOD3WNDNKT,0439886341,1374451200,0.0
3,A1GI0U4ZRJA8WN,0439886341,1334707200,0.0
4,A1QGNMC6O1VW39,0511189877,1397433600,1.0
...,...,...,...,...
4910372,A2YZI3C9MOHC0L,BT008UKTMW,1396569600,1.0
4910373,A322MDK0M89RHN,BT008UKTMW,1313366400,1.0
4910374,A1MH90R0ADMIK0,BT008UKTMW,1404172800,1.0
4910375,A10M2KEFPEQDHN,BT008UKTMW,1297555200,1.0


In [30]:
hot = _drop_cold(sampled_data, 5, 5)

In [31]:
hot.session_id.nunique(), hot.item_id.nunique(), len(hot)

(83427, 29351, 684449)

In [33]:
hot.to_csv("Data/electronics.csv", index=False)

In [37]:
hot.describe()

,session_id,item_id,timestamp,rating
count,684449.000000,684449.000000,6.844490e+05,684449.000000
mean,41662.517719,14758.833548,1.341106e+09,0.809076
std,24092.066264,8385.072620,6.219676e+07,0.393030
min,1.000000,1.000000,9.292320e+08,0.000000
25%,20678.000000,7503.000000,1.318550e+09,1.000000
50%,41627.000000,14824.000000,1.360886e+09,1.000000
75%,62562.000000,21817.000000,1.384992e+09,1.000000
max,83427.000000,29351.000000,1.406074e+09,1.000000
